In [ ]:
from dotenv import load_dotenv
import locale
import numpy as np
import os
import pandas as pd
import sqlalchemy as sa

load_dotenv()

locale.setlocale(locale.LC_ALL, "pt_BR.UTF-8")

engine = sa.create_engine(os.getenv("URL_MYSQL"))

In [ ]:
lista = "02 03 04 05 06 07"
print ("consta") if "08" in lista else print("não consta")

In [ ]:
minhas_apostas = [
    ["05 15 26 27 46 53"],  # aposta n.° 1
    ["03 12 19 20 45 47"],  # aposta n.° 2
    ["01 10 17 41 42 56"],  # aposta n.° 3
    ["02 10 13 27 53 55"],  # aposta n.° 4
    ["06 07 08 11 43 56"],  # aposta n.° 5
    ["08 10 14 25 33 34"],  # aposta n.° 6
    ["05 11 16 40 43 57"],  # aposta n.° 7
    ["04 05 08 13 17 38"],  # aposta n.° 8
    ["13 24 32 49 51 60"],  # aposta n.° 9
    ["11 16 19 43 58 60"],  # aposta n.° 10
    ["03 05 10 20 35 46"],  # aposta n.° 11
    ["02 09 10 19 31 57"],  # aposta n.° 12
    ["04 18 20 21 39 57"],  # aposta n.° 13
    ["02 11 22 36 49 60"],  # aposta n.° 14
    ["02 21 39 48 52 57"],  # aposta n.° 15
    ["14 41 45 50 54 59"],  # aposta n.° 16
    ["13 20 22 25 28 39"],  # aposta n.° 17
    ["01 16 21 34 49 54"],  # aposta n.° 18
]

In [ ]:
new_games = [
    {"concurso": 2747, "data": "2024-07-10", "bola1": 14, "bola2": 17, "bola3": 24, "bola4": 28, "bola5": 36, "bola6": 45, "ganhou": False},
]

df_games = pd.DataFrame(new_games)

rows_games = df_games.to_sql(name="megasena", con=engine, if_exists="append", index=False)
print(f"Foram {rows_games} jogo(s) inserido(s) com sucesso!")

In [ ]:
df_last = pd.read_sql(sql=sa.text("SELECT * FROM megasena"), con=engine)

df_last["concurso"] = df_last["concurso"].astype(str).str.zfill(4)
df_last["data"] = pd.to_datetime(df_last["data"]).dt.strftime("%x (%a)")
df_last["bolas"] = df_last[df_last.columns[2:-1]].apply(lambda x: " ".join(x.astype(str).str.zfill(2)), axis=1)
df_last["ganhou"] = np.where(df_last["ganhou"], "Alguém ganhou", "Ninguém ganhou")
df_last = df_last[["concurso", "data", "bolas", "ganhou"]]
df_last.columns = ["Concurso", "Data", "Bolas", "Bolada?"]

df_last.tail(15)

In [ ]:
stmt = "SELECT * FROM megasena WHERE data IN (SELECT MAX(data) FROM megasena GROUP BY YEAR(data) HAVING YEAR(data) <> YEAR(CURRENT_DATE))"
df_mega_da_virada = pd.read_sql(sql=sa.text(stmt), con=engine)

df_mega_da_virada["concurso"] = df_mega_da_virada["concurso"].astype(str).str.zfill(4)
df_mega_da_virada["data"] = pd.to_datetime(df_mega_da_virada["data"]).dt.strftime("%x (%a)")
df_mega_da_virada["bolas"] = df_mega_da_virada[df_mega_da_virada.columns[2:-1]].apply(lambda x: " ".join(x.astype(str).str.zfill(2)), axis=1)
df_mega_da_virada["ganhou"] = np.where(df_mega_da_virada["ganhou"], "Alguém ganhou", "Ninguém ganhou")
df_mega_da_virada.drop(["bola1", "bola2", "bola3", "bola4", "bola5", "bola6"], axis=1, inplace=True)
df_mega_da_virada = df_mega_da_virada[["concurso", "data", "bolas", "ganhou"]]
df_mega_da_virada.columns = ["Concurso", "Data", "Bolas", "Bolada?"]

df_mega_da_virada